In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
from plotly.offline import iplot
import plotly.io as pio
import random

In [5]:
df.head()

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,NodeID,time_sampled,flow_rate,datetime,volume,day,date,DVol,Date
0,806,806,900949,N01,1.637694e+09,0.02,2021-11-24 00:00:16,0.010,We,2021-11-24,0.010,2021-11-24
1,807,807,900948,N01,1.637694e+09,0.02,2021-11-24 00:00:46,0.010,We,2021-11-24,0.020,2021-11-24
2,808,808,900947,N01,1.637694e+09,0.01,2021-11-24 00:01:16,0.004,We,2021-11-24,0.024,2021-11-24
3,809,809,900946,N01,1.637694e+09,0.00,2021-11-24 00:01:46,0.000,We,2021-11-24,0.024,2021-11-24
4,810,810,900945,N01,1.637694e+09,0.02,2021-11-24 00:02:16,0.012,We,2021-11-24,0.036,2021-11-24


In [ ]:
def add_days(df):
    '''Storing day names for processing each week'''
    dates = list(df['Date'])
    days = []
    for j in dates:
        if j[2] == '/':
            date_object = datetime.strptime(j, '%d/%m/%Y').date()
        else:
            date_object = datetime.strptime(j, '%Y-%m-%d').date()
        days.append(date_object.strftime('%A')[:2])
    df['day'] = days
    return df

## Plots DWU 6 Days

Plotting daily water usage of 6 random days for each node as line graphs.

In [2]:
hh_occ = pd.read_excel('D:\OneDrive - Habib University\HU\KWP\\Data\HH_Occupancy.xlsx')
hh_members = list(hh_occ['Members'])

In [ ]:
'''Faceted Plots'''

weekdays = ['Mo','Tu','We','Th','Fr']
weekends = ['Sa','Su']

for i in range(1,10):
    df = pd.read_csv('D:\OneDrive - Habib University\HU\KWP\\Data\DataDaily\ALLCF_Filtered\ALLCF_Daily'+str(i)+'.csv')
    df = add_days(df) #adding weekdays to data
    df['Hour'] = df.Time.str[:2] #adding col for hour component of Time
    df['Hr-Minute'] = df.Time.str[:5] #adding minute info in sep column, from Time
    no_hh_mem = hh_members[i-1] #no of occupants in household for this node
    # Getting relevant info
    df_week = df[df['day'].isin(weekends)] # only taking datapoints for weekdays
    dates = df_week['Date'].unique() # getting 6 random days to visualise
    rand_dates = random.choices(dates,k=6) 
    df_dates = df_week[df_week["Date"].isin(rand_dates)] #getting datapoints of the 6 selected days only
    df_dates=df_dates.sort_values('Hr-Minute') # sort for x-axis
    # Plotting
    figtitle = 'N0'+str(i)+' - '+str(no_hh_mem)+' Occupants (Weekdays)'
    fig = px.line(df_dates,x='Hr-Minute',y='DVol',facet_row='Date',title=figtitle,height=1000)
    fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1][2:]))
    for annotation in fig['layout']['annotations']: 
        annotation['textangle']= 0
    fig.update_xaxes(categoryorder='array', categoryarray= df_dates['Hr-Minute'].unique())
    fig.show()
    # fig.write_image("Daily Usage\\N0"+str(i)+".png")
    pio.write_image(fig, "Daily Usage\\faceted\\Weekends\\N0"+str(i)+"-6days-weekends.png")

In [ ]:
'''Non-Faceted Plots'''

weekdays = ['Mo','Tu','We','Th','Fr']
weekends = ['Sa','Su']

for i in range(1,27):
    # df = pd.read_csv('D:\OneDrive - Habib University\HU\KWP\\Data\DailyUsage\ALLCF_Useful\ALLCF_Useful'+str(i)+'.csv')
    # df = pd.read_csv('D:\OneDrive - Habib University\HU\KWP\\Data\DailyUsage\ALLCF_Daily0'+str(i)+'.csv')
    df = pd.read_csv('D:\OneDrive - Habib University\HU\KWP\\Data\Preprocessed\\N0'+str(i)+'_cleaned.csv')
    # df = add_days(df)
    df['Hour'] = df.Time.str[:2]
    df['Hr-Minute'] = df.Time.str[:5]
    no_hh_mem = hh_members[i-1]
    # no_hh_mem = '[NA]'
    df_week = df[df['day'].isin(weekends)]
    dates = df_week['Date'].unique()
    if len(dates) < 6:
        continue
    dates_plot = random.choices(dates,k=6)
    df = df.sort_values('Hr-Minute')
    fig = go.Figure()
    for d in dates_plot:
        df_date = df[df["Date"] == d]
        df_date = df_date.sort_values('Hr-Minute') # sort for x-axis
        # ------------
        fig = fig.add_trace(go.Scatter(x = df_date['Hr-Minute'], y = df_date["DVol"], name = str(d)[2:]))
        # add traces to fig and export it
    fig.update_xaxes(categoryorder='array', categoryarray= df['Hr-Minute'].unique())
    fig.update_layout(title='N'+str(i)+' - '+str(no_hh_mem)+' Occupants (Weekends)')
    # fig.show()
    pio.write_image(fig, "Daily Usage - Plots\\useful_days\\Weekends\\N"+str(i)+"-weekends.png")

In [4]:
'''Non-Faceted Plots - Data w/ zeros'''

weekdays = ['Mo','Tu','We','Th','Fr']
weekends = ['Sa','Su']
# summer_months = ['04','05','06','07','08','09'] #april - september
winter_months = ['12','01','02'] #dec - feb

for i in range(1,27):
    df = pd.read_csv('D:\OneDrive - Habib University\HU\KWP\KWP-winter22\Data\Preprocessed\OldSensorData\\filtered-5mins-2000dp\\filtered'+str(i)+'.csv')
    df["Time"] = df.datetime.str[11:]
    df['Hour'] = df.Time.str[:2]
    df['Hr-Minute'] = df.Time.str[:5]
    # df['Date'] = df.datetime.str[:10]
    no_hh_mem = hh_members[i-1]
    df_week = df[df['day'].isin(weekdays)]
    dates = df_week['Date'].unique()
    if len(dates) == 0:
        continue
    elif len(dates) < 6:
        dates_plot = dates
    else:
        dates_plot = random.choices(dates,k=6)
    dates_plot.sort()
    df = df.sort_values('Hr-Minute')
    fig = go.Figure()
    for d in dates_plot:
        df_date = df[df["Date"] == d]
        df_date['DVol'] = df_date['volume'].cumsum()
        df_date = df_date.sort_values('Hr-Minute') # sort for x-axis
        if d[5:7] in winter_months:
            fig = fig.add_trace(go.Scatter(x = df_date['Hr-Minute'], y = df_date["DVol"],mode='lines',line=dict(width=2,dash='dash'), name = str(d)[2:]))
        else:
            fig = fig.add_trace(go.Scatter(x = df_date['Hr-Minute'], y = df_date["DVol"],mode='lines',line=dict(width=2), name = str(d)[2:]))
    fig.update_xaxes(categoryorder='array', categoryarray= df['Hr-Minute'].unique())
    fig.update_traces(marker_size=2)
    fig.update_layout(title='N'+str(i)+' - '+str(no_hh_mem)+' Occupants (Weekdays)',yaxis_title = 'Daily Water Usage (L)', xaxis_title ='Time of Day (h:m)')
    # fig.show()
    pio.write_image(fig, "Daily Usage - Plots\\v4 - with zeros\\filtered-5mins-2000dp-dashed\\N"+str(i)+"-weekdays.png")

C:\Users\Digital Traders\AppData\Local\Temp\ipykernel_7732\505942870.py:28: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Digital Traders\AppData\Local\Temp\ipykernel_7732\505942870.py:28: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Digital Traders\AppData\Local\Temp\ipykernel_7732\505942870.py:28: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http

## Sleeping & Waking 

In [2]:
def get_time_wake(df,threshold=5,ref_win=60):
    '''Returns index of df to indicate when household woke up. Ref window is 30 mins long.'''
    timewake_index = 0
    # Stable region ending ie waking up
    for i in range(1, len(df)):
        increasing = []
        diff = abs(df['DVol'][i] - df['DVol'][i-1])
        # if diff > threshold:
        if diff > 0: # water is being used
            timewake_index = i # people may be waking up now
            if (i + ref_win) < (len(df)-1): # see if people are using water in the succeeding time interval
                window = i + ref_win
            else:
                window = len(df)-1
            for j in range(i,window): # is water being used in next 30 mins
                # if abs(df['DVol'][j] - df['DVol'][j+1]) > threshold:
                if abs(df['DVol'][j] - df['DVol'][j+1]) > 0: # is water being used
                    increasing.append(j)
            if len(increasing) < (0.5*window): # if water is not being used > 30% of the window, people aren't awake yet
                timewake_index = 0 # resetting
            else:
                break
    return timewake_index

In [3]:
def get_time_sleep(df,threshold=5,ref_win=30):
    '''Returns index of df to indicate when household sleeps. Ref window is defaulted at 30 mins long.'''
    # Stable start ie going to sleep
    timesleep_index = len(df)-1
    for i in range(1, len(df)):
        sleeping = []
        # if abs(df['DVol'][i] - df['DVol'][i-1]) < threshold:
        if abs(df['DVol'][i] - df['DVol'][i-1]) == 0: # water is not being used
            timesleep_index = i
            if (i + ref_win) < (len(df)-1):
                window = i + ref_win
            else:
                window = len(df)-1
            for j in range(i,window):
                # if abs(df['DVol'][j] - df['DVol'][j+1]) < threshold:
                if abs(df['DVol'][j] - df['DVol'][j+1]) == 0: # water is not being used
                    sleeping.append(j)
            if len(sleeping) < (0.8*window): # if water is being used > 20% of the window, people aren't asleep yet 
                timesleep_index = len(df)-1 #resetting
            else: # if water usage is 0 for more than 80% of the window, people may be going to sleep now, so stop loop and return current index
                break
    return timesleep_index

In [7]:
'''For each day/date, group data by minute and calculate when the household woke up and went to sleep on that day. Add these values of each day to new dataframe.'''

for i in range(1,2): # for each node 
    # df = pd.read_csv('D:\OneDrive - Habib University\HU\KWP\\Data\DataDaily\ALLCF_Daily0'+str(i)+'.csv')
    df = pd.read_csv('D:\OneDrive - Habib University\HU\KWP\KWP-winter22\\Data\Preprocessed\OldSensorData\\filtered-5mins-2000dp\\filtered'+str(i)+'.csv')
    dates = df['Date'].unique()
    wake_sleep_times = pd.DataFrame(columns = ['Node','Date','TimeWake','TimeSleep'])
    df['Time'] = df.datetime.str[11:]
    df['Hr-Minute'] = df.Time.str[:5]
    # threshold = 5
    for date in dates: #for each date of the node
        df_date = df[df["Date"] == date]
        df_date = df_date.groupby('Hr-Minute').max() #cumulative, next hour already includes prev hour in it, can use just Hour since its one day
        # # ------------
        # fig = px.line(df_date,x=df_date.index,y='DVol')
        # fig.show()
        # break
        # # ------------
        timesleep_index = get_time_sleep(df_date)
        timewake_index = get_time_wake(df_date)
        # Add new row to final df
        new_row = [df_date['NodeID'][timewake_index], date, df_date['Time'][timewake_index], df_date['Time'][timesleep_index]]
        wake_sleep_times.loc[len(wake_sleep_times)] = new_row
    wake_sleep_times.to_csv('D:\OneDrive - Habib University\HU\KWP\KWP-winter22\\Data\SleepingPatterns/SP_N'+str(i)+'.csv')
    # wake_sleep_times.head()
    # break


In [12]:
df = pd.read_csv('D:\OneDrive - Habib University\HU\KWP\KWP-winter22\\Data\SleepingPatterns/SP_N'+str(3)+'.csv')
# df = df.sort_values('TimeSleep')
fig = px.line(df,x='Date',y=['TimeWake','TimeSleep'])
# fig.update_yaxes(categoryorder='array', categoryarray= df['TimeSleep'].unique())
fig.show()

## Mov Avg

In [ ]:
fig = go.Figure()
for i in range(2,28):
    df = pd.read_csv('D:\OneDrive - Habib University\HU\KWP\\Data\DataDaily\ALLTF_Daily '+str(i)+' .csv') #2-27
    # df['date'] = df.datetime.str[:10]
    df['mov_avg'] = df['TotalVol'].rolling(window=30).mean()
    fig = fig.add_trace(go.Scatter(x = df['Date'], y = df["mov_avg"], name = 'N'+str(i-1)))
fig.update_layout(title='Monthly Moving Average')
fig.show()

In [ ]:
fig = px.line(df,x='Date',y=['mov_avg'])
fig.show()

## Useful Data

In [ ]:
'''Adding #datapoints to files with total water usage/day (TF)'''
for i in range(10,27):
    df_cf = pd.read_csv('D:\OneDrive - Habib University\HU\KWP\\Data\DataDaily\ALLCF_Daily'+str(i)+'.csv') #1-26
    df_tf = pd.read_csv('D:\OneDrive - Habib University\HU\KWP\\Data\DataDaily\ALLTF_Daily '+str(i+1)+' .csv') #2-27
    dates = df_cf['Date'].unique()
    dtpts = []
    for date in dates: #for each date of the node
        df_date = df_cf[df_cf["Date"] == date]
        no_dtpts = len(df_date)
        dtpts.append(no_dtpts)
    df_tf['#Datapoints'] = dtpts
    df_tf.to_csv('D:\OneDrive - Habib University\HU\KWP\\Data\DataDaily\ALLTF\ALLTF_Daily'+str(i)+'.csv')

In [ ]:
'''More info on #datapoints/day for each node.'''
# c_2880 = []
c_L1000 = []
# c_G2880 = []
for i in range(1,27):
    df_tf = pd.read_csv('D:\OneDrive - Habib University\HU\KWP\\Data\DataDaily\ALLTF_Daily'+str(i)+'.csv') #1-26
    # c_2880.append(list(df_tf['#Datapoints']).count(2880))
    c_L1000.append(len(df_tf[df_tf['#Datapoints']<1000]))
    # c_G2880.append(len(df_tf[df_tf['#Datapoints']>2880]))
# print(len(c_2880))
print(len(c_L1000))
# print(len(c_G2880))
df = pd.read_csv('D:\OneDrive - Habib University\HU\KWP\\Data\DataDaily\ALLCF_DataPointsPerDay.csv')
# df['=2880'] = c_2880
# df['<200'] = c_L200
# df['>2880'] = c_G2880
df['<1000 (#days)'] = c_L1000
df.to_csv('D:\OneDrive - Habib University\HU\KWP\\Data\DataDaily\ALLCF_DataPointsPerDay.csv')

'''No. of days in raw data vs in preprocessed data.'''


In [ ]:
days = pd.DataFrame(columns=['NodeID','Raw','Preprocessed-w0s','Preprocessed-wo0s'])

In [ ]:
'''No. of days in raw data vs in preprocessed data.'''
for i in range(10,27):
    row = []
    df = pd.read_csv('D:\OneDrive - Habib University\HU\KWP\\Data\ALLSENSORDATA\\N'+str(i)+'.csv') #1-26
    time_sampled = list(df['time_sampled'])
    datetime_obj = []
    for j in time_sampled:
        datetime_obj.append(str(datetime.fromtimestamp(int(j)))) #converts each timestamp to datetime format using computer's local timezone ie KHI
    df['datetime'] = datetime_obj
    df['date'] = df.datetime.str[0:10]
    dates = len(df['date'].unique())
    row.append('N'+str(i))
    row.append(dates)
    df = pd.read_csv('D:\OneDrive - Habib University\HU\KWP\\Data\Preprocessed\\N'+str(i)+'_cleaned.csv') #1-26
    df['date'] = df.datetime.str[0:10]
    dates = len(df['date'].unique())
    row.append(dates)
    df = pd.read_csv('D:\OneDrive - Habib University\HU\KWP\\Data\DailyUsage\ALLTF_Daily'+str(i)+'.csv') #1-26
    row.append(len(df))
    days.loc[len(days)] = row

In [ ]:
'''Appending #days for new dataset to existing file'''
no_days = []
for i in range(1,27):
    df = pd.read_csv('D:\OneDrive - Habib University\HU\KWP\\Data\Preprocessed\OldSensorData\\with dvol\\N'+str(i)+'_cleaned.csv')
    df['Date'] = df.datetime.str[0:10]
    no_days.append(len(df['Date'].unique()))
days = pd.read_csv('D:\OneDrive - Habib University\HU\KWP\\Data\\No.DaysinDataset.csv')
days['WithZ'] = no_days
days.to_csv('D:\OneDrive - Habib University\HU\KWP\\Data\\No.DaysinDataset.csv')

Quantiles for volume and no. datapoints

In [ ]:
'''Freq distribution of total daily volume/datapoints for each node'''
vol_quantiles = pd.DataFrame(columns=['NodeID','5th','10th','50th','90th','95th'])
for i in range(1,27):
    row = []
    # df = pd.read_csv('D:\OneDrive - Habib University\HU\KWP\\Data\DailyUsage\ALLTF_Daily'+str(i)+'.csv') #1-26
    df = pd.read_csv('D:\OneDrive - Habib University\HU\KWP\\Data\Preprocessed\TF_N'+str(i)+'.csv') #1-26
    df['no_dtpts'] = df['TotalVol']
    row = ['N'+str(i), df.no_dtpts.quantile(0.05),df.no_dtpts.quantile(0.1),df.no_dtpts.quantile(0.5),df.no_dtpts.quantile(0.9),df.no_dtpts.quantile(0.95)]
    vol_quantiles.loc[len(vol_quantiles)] = row
    # fig = px.histogram(df, x="no_dtpts", nbins=50,title='N'+str(i))
    # # fig.show()
    # pio.write_image(fig, 'D:\OneDrive - Habib University\HU\KWP\\Figures\DP-Quant\DailyDP-N'+str(i)+'.png')
vol_quantiles.to_csv('D:\OneDrive - Habib University\HU\KWP\\Data\\VolQuantiles-withzeros.csv')

## removing days with missing data

In [ ]:
def five_mins_missing(df):
    '''Takes dataframe for one day as input. Returns True if more than 5mins of consecutive data is missing in dataframe.'''
    df['datetime'] = df['datetime'].apply(lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S"))
    ser_diff = df['datetime'].diff()
    ser_diff = ser_diff.dt.total_seconds().div(60, fill_value=0)
    if ser_diff.max() > 5: #if max time difference in datapoints in > 5 minutes
        return True
    return False

In [ ]:
for i in range(1,27): # for each node 
    print('Node',i)
    df = pd.read_csv('D:\OneDrive - Habib University\HU\KWP\\Data\Preprocessed\OldSensorData\with dvol\\N'+str(i)+'_cleaned.csv') #1-26
    df_tf = pd.read_csv('D:\OneDrive - Habib University\HU\KWP\\Data\Preprocessed\OldSensorData\TF_N'+str(i)+'.csv') #1-26
    MIN_DTPTS = 2870 #df_tf.No_dp.quantile(0.99) 
    MIN_DVOL = 200 if df_tf.TotalVol.quantile(0.1) < 20 else df_tf.TotalVol.quantile(0.1) #10th percentile or 200 (if low perc.)
    df['Date'] = df.datetime.str[:10]
    dates = df['Date'].unique()
    for date in dates: #for each date of the node
        df_date = df[df["Date"] == date]
        if len(df_date) < MIN_DTPTS or list(df_date['DVol'])[-1] < MIN_DVOL: #len(df_date) < MIN_DTPTS
            df = df[df["Date"] != date] #remove datapoints from this date from dataset
    # path = "".join('D:\OneDrive - Habib University\HU\KWP\\Data\Preprocessed\useful days\\useful_N',str(i),'.csv')
    # df.to_csv(path)
    df.to_csv('D:\OneDrive - Habib University\HU\KWP\\Data\Preprocessed\OldSensorData\\filtered-2870dp\\filtered'+str(i)+'.csv')
# df.head()


## Peak WU

In [ ]:
peak = pd.DataFrame(columns=['NodeID','total_days','peak_days','total_vol','peak_vol','occupants'])
for i in range(1,27):
    df = pd.read_csv('D:\OneDrive - Habib University\HU\KWP\\Data\DailyUsage\ALLTF_Daily'+str(i)+'.csv') #1-26
    df_peak = df[df['TotalVol'] > df.TotalVol.quantile(0.90)]
    no_hh_mem = hh_members[i-1]
    row = ['N'+str(i),len(df),len(df_peak),df['TotalVol'].sum(),df_peak['TotalVol'].sum(),no_hh_mem]
    peak.loc[len(peak)]= row
    # fig = px.line(df, x='Date', y = 'TotalVol', title='N'+str(i)+' ,Occupants: '+str(no_hh_mem))
    # pio.write_image(fig, 'D:\OneDrive - Habib University\HU\KWP\\Figures\Peak Usage\Peak-N'+str(i)+'.png')
peak.to_csv('D:\OneDrive - Habib University\HU\KWP\\Data\PeakWU.csv') #1-26
    

## WU - Mor, Aft, Eve.

In [ ]:
period_day = pd.DataFrame(columns=['NodeID','mornings_tot(%)','afternoons_tot(%)','evenings_tot(%)'])
nodes = []

In [ ]:
morns = []
noons = []
eve = []

In [ ]:
for i in range(10,27):
    df = pd.read_csv('D:\OneDrive - Habib University\HU\KWP\\Data\DailyUsage\ALLCF_Daily'+str(i)+'.csv') #1-26
    df_tf = pd.read_csv('D:\OneDrive - Habib University\HU\KWP\\Data\DailyUsage\ALLTF_Daily'+str(i)+'.csv') #1-26
    df['Hour'] = df.Time.str[:2]
    df_morn = df[df['Hour'].isin(['04','05','06','07','08','09','10','11'])].groupby('Date').max()
    df_eve = df[df['Hour'].isin(['17','18','19','20','21','22','23'])].groupby('Date').max()
    df_aftnoon = df[df['Hour'].isin(['12','13','14','15','16'])].groupby('Date').max()

    # subtracting volume of previous period of day, as dvol is cumulative daily volume
    df_eve['DVol'] = df_eve['DVol'] - df_aftnoon['DVol']
    df_aftnoon['DVol'] = df_aftnoon['DVol'] - df_morn['DVol']

    # #total vol used by node in period of day as % of total vol used by node
    # morns.append(df_morn['DVol'].sum()/df_tf['TotalVol'].sum()*100) 
    # noons.append(df_aftnoon['DVol'].sum()/df_tf['TotalVol'].sum()*100)
    # eve.append(df_eve['DVol'].sum()/df_tf['TotalVol'].sum()*100)
    
    # # plotting figures for each node's usage during each period of day
    # fig = px.line(df_aftnoon,x=df_aftnoon.index,y='DVol',title='N'+str(i)+' - Afternoons')
    # pio.write_image(fig, 'D:\OneDrive - Habib University\HU\KWP\\Figures\Afternoons\\afternoons-N'+str(i)+'.png')
    # fig = px.line(df_eve,x=df_eve.index,y='DVol',title='N'+str(i)+' - Evenings')
    # pio.write_image(fig, 'D:\OneDrive - Habib University\HU\KWP\\Figures\Evenings\\evenings-N'+str(i)+'.png')

    # Plotting all time periods of each node on same figure
    fig = go.Figure()
    fig = fig.add_trace(go.Scatter(x = df_morn.index, y = df_morn["DVol"], name = 'Mornings'))
    fig = fig.add_trace(go.Scatter(x = df_aftnoon.index, y = df_aftnoon["DVol"], name = 'Afternoons'))
    fig = fig.add_trace(go.Scatter(x = df_eve.index, y = df_eve["DVol"], name = 'Evenings'))
    fig.update_layout(title='N'+str(i)+' - '+str(hh_members[i-1])+' Occupants (Weekends)')
    pio.write_image(fig, 'D:\OneDrive - Habib University\HU\KWP\\Figures\\timesofday\\periods-N'+str(i)+'.png')

In [ ]:
period_day['mornings_tot(%)'] = morns
period_day['afternoons_tot(%)'] = noons
period_day['evenings_tot(%)'] = eve
period_day.to_csv('D:\OneDrive - Habib University\HU\KWP\\Data\PeriodsWU_v2.csv')

## Window - 50% WU

In [ ]:
def smallestSubWithSum(arr, n, x):
	# Initialize length of smallest subarray as n+1
	min_len = n + 1
	# Pick every element as starting point
	for start in range(0,n):
		# Initialize sum startingwith current start
		curr_sum = arr[start]
		# If first element itself is greater
		if (curr_sum > x):
			return 1
		# Try different ending points for current start
		for end in range(start+1,n):
			# add last element to current sum
			curr_sum += arr[end]
			# If sum becomes more than x and length of this subarray is smaller than current smallest length, update the smallest length (or result)
			if curr_sum > x and (end - start + 1) < min_len:
				min_len = (end - start + 1)
	return min_len


In [2]:
def min_sublist(lst, value):
    min_length = float('inf')
    start = end = 0
    curr_sum = 0
    
    for i, num in enumerate(lst):
        curr_sum += num
        end = i
        
        while curr_sum >= value:
            if end - start + 1 < min_length:
                min_length = end - start + 1
                min_start = start
                min_end = end
            curr_sum -= lst[start]
            start += 1
    
    return min_start, min_end

# # Test the function
# lst = [1,8, 2, 3, 4, 5, 4]
# value = 9
# print(min_sublist(lst, value))  # Output: (2, 4)


In [3]:
i = 1
fifty = pd.DataFrame(columns=['NodeID','date','time_start','time_end','50WU'])
df = pd.read_csv('D:\OneDrive - Habib University\HU\KWP\KWP-winter22\\Data\Preprocessed\OldSensorData\with dvol\\N'+str(i)+'_cleaned.csv') #1-26
df['Date'] = df.datetime.str[:10]
dates = df['Date'].unique()
for d in dates: #for each date of the node
    df_date = df[df["Date"] == d] 
    fifty_date = df_date['volume'].sum()*0.5
    if fifty_date > 0:
        w_start, w_end = min_sublist(list(df_date['volume']),fifty_date)
        time_start = list(df_date['datetime'])[w_start]
        time_end = list(df_date['datetime'])[w_end]
        fifty.loc[len(fifty)] = ['N'+str(i),d,time_start,time_end,fifty_date]
    else:
        fifty.loc[len(fifty)] = ['N'+str(i),d,0,0,fifty_date]
fifty.to_csv('D:\OneDrive - Habib University\HU\KWP\\Data\50pWU\\N'+str(i)+'.csv')

KeyboardInterrupt: 